# Libraries
Pandas, Matplotlib, Numpy, Sklearn

Pandas

In [4]:
import pandas as pd

Matplolib

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
from pprint import pprint

Sklearn

In [2]:
from sklearn import svm, datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, confusion_matrix
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.utils.multiclass import unique_labels

Numpy

In [5]:
import numpy as np
np.set_printoptions(precision=2)

# Data
Cleaning & Descriptive Statistics

## Data Import

Importing

In [6]:
url = 'https://drive.google.com/file/d/1wk5pql6ML6jIi1h7aIS3bIUVZ9M3E5xs/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
df = pd.read_csv(path)
df.sample(5)

,lnpercapitaconsumption,d_fuel_other,d_fuel_wood,d_fuel_coal,d_fuel_kerosene,d_fuel_gas,d_fuel_electric,d_fuel_none,d_water_other,d_water_river,...,d_lux_4,d_lux_5,training,percapitaconsumption,poor,h_hhsize,id_for_matlab,hhid,lncaphat_OLS,percapitahat_OLS
10740,NaN,0,0,0,0,1,0,0,0,0,...,0,0,0,NaN,0,5,10741,324400111,5.876216,382.4636
5451,NaN,0,0,0,0,1,0,0,0,0,...,0,0,0,NaN,0,4,5452,145102911,5.458727,213.5435
43148,5.977211,0,0,0,0,1,0,0,0,0,...,0,0,1,394.3389,0,9,43149,201707711,6.314787,696.4991
17788,NaN,0,0,0,0,1,0,0,0,0,...,0,0,0,NaN,0,1,17789,320403011,6.486506,759.6255
33332,7.172359,0,0,1,0,0,0,0,0,0,...,0,0,1,1302.9150,0,1,33333,242508711,6.265194,580.1896


## Data Processing

In [40]:
df.shape

(46305, 81)

In [41]:
# for later use 
df_later = df[['training', 'percapitaconsumption', 'poor', 'h_hhsize','id_for_matlab', 'hhid', 'lncaphat_OLS', 'percapitahat_OLS']]

In [43]:

df_later = df_later[:23152]

In [44]:
df_later.shape

(23152, 8)

In [45]:
df_later.head()

,training,percapitaconsumption,poor,h_hhsize,id_for_matlab,hhid,lncaphat_OLS,percapitahat_OLS
0,0,NaN,0,1,1,233710511,5.246471,284.4243
1,0,NaN,0,3,2,295508011,6.129468,522.8840
2,0,NaN,0,4,3,257600211,6.665577,878.4961
3,0,NaN,1,9,4,198104311,6.088295,567.4705
4,0,NaN,1,8,5,24805311,5.008170,115.4929


Not taking dummy-variable trap into consideration. --> Removing everything after column "training"

In [7]:
df = df.drop(['training', 'percapitaconsumption', 'poor', 'h_hhsize','id_for_matlab', 'hhid', 'lncaphat_OLS', 'percapitahat_OLS'], axis=1)

Identifying NaN

Since most of the NaN represent the outcome variable, it does not make sense to keep these observations. Hence, we are dropping all observations containing any kind of NaN. 

In [8]:
print("Before dropping NAs:")
print(df.shape)
# creating prediction set before dropping NA
dfPred = df[:23152]
print("\nPrediction Set:")
print(dfPred.shape)
# dropping NA
df = df.dropna() 
print("\nTraining Set:")
print(df.shape)

Before dropping NAs:
(46305, 73)

Prediction Set:
(23152, 73)

Training Set:
(22674, 73)


Setting variables for Prediction Set

# Machine Learning Technique
Data Splitting & Machine Learning Algorithms

## Data Splitting

Training Data Partitioning 
- 50% Train
- 50% Validation

In [9]:
X = df.iloc[:, df.columns!="lnpercapitaconsumption"]
y = df.loc[:, "lnpercapitaconsumption"]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .5 , random_state=4)

print("Dimensions of the train set: ")
print(X_train.shape)
print(y_train.shape)
print("")
print("Dimensions of the test set: ")
print(X_test.shape)
print(y_test.shape)

Dimensions of the train set: 
(11337, 72)
(11337,)

Dimensions of the test set: 
(11337, 72)
(11337,)


In [11]:
# function for model evaluation
def evaluate(model, test_features, test_labels, y_pred):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print("Model Performance:\n")
    print("Accuracy: {:0.2f}%".format(accuracy))
    print("Mean Absolute Error:", mean_absolute_error(test_labels, y_pred))
    print("Mean Squared Error:", mean_squared_error(test_labels, y_pred))
    print("Root Mean Squared Error:", np.sqrt(mean_squared_error(test_labels, y_pred)))
    return

In [10]:
rf_maxDepth= RandomForestRegressor(n_estimators = 1500, min_samples_leaf=1, min_samples_split = 11, random_state=4, verbose=3)
rf_maxDepth.fit(X_train,y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s


building tree 1 of 1500
building tree 2 of 1500
building tree 3 of 1500
building tree 4 of 1500
building tree 5 of 1500
building tree 6 of 1500
building tree 7 of 1500
building tree 8 of 1500
building tree 9 of 1500
building tree 10 of 1500
building tree 11 of 1500
building tree 12 of 1500
building tree 13 of 1500
building tree 14 of 1500
building tree 15 of 1500
building tree 16 of 1500
building tree 17 of 1500
building tree 18 of 1500
building tree 19 of 1500
building tree 20 of 1500
building tree 21 of 1500
building tree 22 of 1500
building tree 23 of 1500
building tree 24 of 1500
building tree 25 of 1500
building tree 26 of 1500
building tree 27 of 1500
building tree 28 of 1500
building tree 29 of 1500
building tree 30 of 1500
building tree 31 of 1500
building tree 32 of 1500
building tree 33 of 1500
building tree 34 of 1500
building tree 35 of 1500
building tree 36 of 1500
building tree 37 of 1500
building tree 38 of 1500
building tree 39 of 1500
building tree 40 of 1500
building 

building tree 323 of 1500
building tree 324 of 1500
building tree 325 of 1500
building tree 326 of 1500
building tree 327 of 1500
building tree 328 of 1500
building tree 329 of 1500
building tree 330 of 1500
building tree 331 of 1500
building tree 332 of 1500
building tree 333 of 1500
building tree 334 of 1500
building tree 335 of 1500
building tree 336 of 1500
building tree 337 of 1500
building tree 338 of 1500
building tree 339 of 1500
building tree 340 of 1500
building tree 341 of 1500
building tree 342 of 1500
building tree 343 of 1500
building tree 344 of 1500
building tree 345 of 1500
building tree 346 of 1500
building tree 347 of 1500
building tree 348 of 1500
building tree 349 of 1500
building tree 350 of 1500
building tree 351 of 1500
building tree 352 of 1500
building tree 353 of 1500
building tree 354 of 1500
building tree 355 of 1500
building tree 356 of 1500
building tree 357 of 1500
building tree 358 of 1500
building tree 359 of 1500
building tree 360 of 1500
building tre

building tree 642 of 1500
building tree 643 of 1500
building tree 644 of 1500
building tree 645 of 1500
building tree 646 of 1500
building tree 647 of 1500
building tree 648 of 1500
building tree 649 of 1500
building tree 650 of 1500
building tree 651 of 1500
building tree 652 of 1500
building tree 653 of 1500
building tree 654 of 1500
building tree 655 of 1500
building tree 656 of 1500
building tree 657 of 1500
building tree 658 of 1500
building tree 659 of 1500
building tree 660 of 1500
building tree 661 of 1500
building tree 662 of 1500
building tree 663 of 1500
building tree 664 of 1500
building tree 665 of 1500
building tree 666 of 1500
building tree 667 of 1500
building tree 668 of 1500
building tree 669 of 1500
building tree 670 of 1500
building tree 671 of 1500
building tree 672 of 1500
building tree 673 of 1500
building tree 674 of 1500
building tree 675 of 1500
building tree 676 of 1500
building tree 677 of 1500
building tree 678 of 1500
building tree 679 of 1500
building tre

building tree 961 of 1500
building tree 962 of 1500
building tree 963 of 1500
building tree 964 of 1500
building tree 965 of 1500
building tree 966 of 1500
building tree 967 of 1500
building tree 968 of 1500
building tree 969 of 1500
building tree 970 of 1500
building tree 971 of 1500
building tree 972 of 1500
building tree 973 of 1500
building tree 974 of 1500
building tree 975 of 1500
building tree 976 of 1500
building tree 977 of 1500
building tree 978 of 1500
building tree 979 of 1500
building tree 980 of 1500
building tree 981 of 1500
building tree 982 of 1500
building tree 983 of 1500
building tree 984 of 1500
building tree 985 of 1500
building tree 986 of 1500
building tree 987 of 1500
building tree 988 of 1500
building tree 989 of 1500
building tree 990 of 1500
building tree 991 of 1500
building tree 992 of 1500
building tree 993 of 1500
building tree 994 of 1500
building tree 995 of 1500
building tree 996 of 1500
building tree 997 of 1500
building tree 998 of 1500
building tre

building tree 1268 of 1500
building tree 1269 of 1500
building tree 1270 of 1500
building tree 1271 of 1500
building tree 1272 of 1500
building tree 1273 of 1500
building tree 1274 of 1500
building tree 1275 of 1500
building tree 1276 of 1500
building tree 1277 of 1500
building tree 1278 of 1500
building tree 1279 of 1500
building tree 1280 of 1500
building tree 1281 of 1500
building tree 1282 of 1500
building tree 1283 of 1500
building tree 1284 of 1500
building tree 1285 of 1500
building tree 1286 of 1500
building tree 1287 of 1500
building tree 1288 of 1500
building tree 1289 of 1500
building tree 1290 of 1500
building tree 1291 of 1500
building tree 1292 of 1500
building tree 1293 of 1500
building tree 1294 of 1500
building tree 1295 of 1500
building tree 1296 of 1500
building tree 1297 of 1500
building tree 1298 of 1500
building tree 1299 of 1500
building tree 1300 of 1500
building tree 1301 of 1500
building tree 1302 of 1500
building tree 1303 of 1500
building tree 1304 of 1500
b

[Parallel(n_jobs=1)]: Done 1500 out of 1500 | elapsed:  1.8min finished


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=11, min_weight_fraction_leaf=0.0,
                      n_estimators=1500, n_jobs=None, oob_score=False,
                      random_state=4, verbose=3, warm_start=False)

In [ ]:
rf_lessDepth= RandomForestRegressor(n_estimators = 1900, max_depth = 48, min_samples_leaf = 2, min_samples_split = 11, random_state=4, verbose=3)
rf_lessDepth.fit(X_train,y_train)

In [12]:
y_pred_maxDepth = rf_maxDepth.predict(X_test)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1500 out of 1500 | elapsed:    4.0s finished


In [ ]:
y_pred_maxDepth = rf_maxDepth.predict(X_test)
y_pred_lessDepth = rf_lessDepth.predict(X_test)

In [13]:
rf_maxDepth_accuracy = evaluate(rf_maxDepth, X_test, y_test, y_pred_maxDepth)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


Model Performance:

Accuracy: 93.87%
Mean Absolute Error: 0.34659415038266606
Mean Squared Error: 0.2005277801589417
Root Mean Squared Error: 0.4478032828809339


[Parallel(n_jobs=1)]: Done 1500 out of 1500 | elapsed:    3.9s finished


In [ ]:
rf_lesstDepth_accuracy = evaluate(rf_lessDepth, X_test, y_test, y_pred_lessDepth)

### Training on complete Training Set & Predict Values

In [25]:
dfPred = pd.DataFrame(dfPred).fillna(0)

In [26]:
XPred = dfPred.iloc[:, df.columns!="lnpercapitaconsumption"]
yPred = dfPred.loc[:, "lnpercapitaconsumption"]

In [10]:
rf_Final= RandomForestRegressor(n_estimators = 1500, min_samples_leaf=1, min_samples_split = 11, random_state=4, verbose=3)
rf_Final.fit(X,y)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


building tree 1 of 1500
building tree 2 of 1500


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s


building tree 3 of 1500
building tree 4 of 1500
building tree 5 of 1500
building tree 6 of 1500
building tree 7 of 1500
building tree 8 of 1500
building tree 9 of 1500
building tree 10 of 1500
building tree 11 of 1500
building tree 12 of 1500
building tree 13 of 1500
building tree 14 of 1500
building tree 15 of 1500
building tree 16 of 1500
building tree 17 of 1500
building tree 18 of 1500
building tree 19 of 1500
building tree 20 of 1500
building tree 21 of 1500
building tree 22 of 1500
building tree 23 of 1500
building tree 24 of 1500
building tree 25 of 1500
building tree 26 of 1500
building tree 27 of 1500
building tree 28 of 1500
building tree 29 of 1500
building tree 30 of 1500
building tree 31 of 1500
building tree 32 of 1500
building tree 33 of 1500
building tree 34 of 1500
building tree 35 of 1500
building tree 36 of 1500
building tree 37 of 1500
building tree 38 of 1500
building tree 39 of 1500
building tree 40 of 1500
building tree 41 of 1500
building tree 42 of 1500
buildin

building tree 323 of 1500
building tree 324 of 1500
building tree 325 of 1500
building tree 326 of 1500
building tree 327 of 1500
building tree 328 of 1500
building tree 329 of 1500
building tree 330 of 1500
building tree 331 of 1500
building tree 332 of 1500
building tree 333 of 1500
building tree 334 of 1500
building tree 335 of 1500
building tree 336 of 1500
building tree 337 of 1500
building tree 338 of 1500
building tree 339 of 1500
building tree 340 of 1500
building tree 341 of 1500
building tree 342 of 1500
building tree 343 of 1500
building tree 344 of 1500
building tree 345 of 1500
building tree 346 of 1500
building tree 347 of 1500
building tree 348 of 1500
building tree 349 of 1500
building tree 350 of 1500
building tree 351 of 1500
building tree 352 of 1500
building tree 353 of 1500
building tree 354 of 1500
building tree 355 of 1500
building tree 356 of 1500
building tree 357 of 1500
building tree 358 of 1500
building tree 359 of 1500
building tree 360 of 1500
building tre

building tree 640 of 1500
building tree 641 of 1500
building tree 642 of 1500
building tree 643 of 1500
building tree 644 of 1500
building tree 645 of 1500
building tree 646 of 1500
building tree 647 of 1500
building tree 648 of 1500
building tree 649 of 1500
building tree 650 of 1500
building tree 651 of 1500
building tree 652 of 1500
building tree 653 of 1500
building tree 654 of 1500
building tree 655 of 1500
building tree 656 of 1500
building tree 657 of 1500
building tree 658 of 1500
building tree 659 of 1500
building tree 660 of 1500
building tree 661 of 1500
building tree 662 of 1500
building tree 663 of 1500
building tree 664 of 1500
building tree 665 of 1500
building tree 666 of 1500
building tree 667 of 1500
building tree 668 of 1500
building tree 669 of 1500
building tree 670 of 1500
building tree 671 of 1500
building tree 672 of 1500
building tree 673 of 1500
building tree 674 of 1500
building tree 675 of 1500
building tree 676 of 1500
building tree 677 of 1500
building tre

building tree 957 of 1500
building tree 958 of 1500
building tree 959 of 1500
building tree 960 of 1500
building tree 961 of 1500
building tree 962 of 1500
building tree 963 of 1500
building tree 964 of 1500
building tree 965 of 1500
building tree 966 of 1500
building tree 967 of 1500
building tree 968 of 1500
building tree 969 of 1500
building tree 970 of 1500
building tree 971 of 1500
building tree 972 of 1500
building tree 973 of 1500
building tree 974 of 1500
building tree 975 of 1500
building tree 976 of 1500
building tree 977 of 1500
building tree 978 of 1500
building tree 979 of 1500
building tree 980 of 1500
building tree 981 of 1500
building tree 982 of 1500
building tree 983 of 1500
building tree 984 of 1500
building tree 985 of 1500
building tree 986 of 1500
building tree 987 of 1500
building tree 988 of 1500
building tree 989 of 1500
building tree 990 of 1500
building tree 991 of 1500
building tree 992 of 1500
building tree 993 of 1500
building tree 994 of 1500
building tre

building tree 1263 of 1500
building tree 1264 of 1500
building tree 1265 of 1500
building tree 1266 of 1500
building tree 1267 of 1500
building tree 1268 of 1500
building tree 1269 of 1500
building tree 1270 of 1500
building tree 1271 of 1500
building tree 1272 of 1500
building tree 1273 of 1500
building tree 1274 of 1500
building tree 1275 of 1500
building tree 1276 of 1500
building tree 1277 of 1500
building tree 1278 of 1500
building tree 1279 of 1500
building tree 1280 of 1500
building tree 1281 of 1500
building tree 1282 of 1500
building tree 1283 of 1500
building tree 1284 of 1500
building tree 1285 of 1500
building tree 1286 of 1500
building tree 1287 of 1500
building tree 1288 of 1500
building tree 1289 of 1500
building tree 1290 of 1500
building tree 1291 of 1500
building tree 1292 of 1500
building tree 1293 of 1500
building tree 1294 of 1500
building tree 1295 of 1500
building tree 1296 of 1500
building tree 1297 of 1500
building tree 1298 of 1500
building tree 1299 of 1500
b

[Parallel(n_jobs=1)]: Done 1500 out of 1500 | elapsed:  3.8min finished


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=11, min_weight_fraction_leaf=0.0,
                      n_estimators=1500, n_jobs=None, oob_score=False,
                      random_state=4, verbose=3, warm_start=False)

In [27]:
y_pred_Final = rf_Final.predict(XPred)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1500 out of 1500 | elapsed:    8.5s finished


In [95]:
Predictions = pd.DataFrame(y_pred_Final, columns=["lnpercapitaconsumption"])

In [96]:
Predictions.to_csv('lnPCC_Pred.csv')

In [97]:
df_Figure5 = Predictions.join(df_later, how= "right")

In [98]:
df_Figure5.head()

,lnpercapitaconsumption,training,percapitaconsumption,poor,h_hhsize,id_for_matlab,hhid,lncaphat_OLS,percapitahat_OLS
0,5.217372,0,NaN,0,1,1,233710511,5.246471,284.4243
1,6.127908,0,NaN,0,3,2,295508011,6.129468,522.8840
2,6.749911,0,NaN,0,4,3,257600211,6.665577,878.4961
3,6.139218,0,NaN,1,9,4,198104311,6.088295,567.4705
4,5.003568,0,NaN,1,8,5,24805311,5.008170,115.4929


In [100]:
lncpc = df_Figure5["lnpercapitaconsumption"]

In [101]:
new_values =  np.exp(lncpc)

In [102]:
new_values

0         184.448727
1         458.475960
2         853.983038
3         463.690909
4         148.943714
            ...     
23147     162.990426
23148     142.149571
23149     165.069305
23150    1160.764593
23151     352.958315
Name: lnpercapitaconsumption, Length: 23152, dtype: float64

In [103]:
df_Figure5["percapitaconsumption"] = new_values

In [105]:
df_Figure5.head()

,lnpercapitaconsumption,training,percapitaconsumption,poor,h_hhsize,id_for_matlab,hhid,lncaphat_OLS,percapitahat_OLS
0,5.217372,0,184.448727,0,1,1,233710511,5.246471,284.4243
1,6.127908,0,458.475960,0,3,2,295508011,6.129468,522.8840
2,6.749911,0,853.983038,0,4,3,257600211,6.665577,878.4961
3,6.139218,0,463.690909,1,9,4,198104311,6.088295,567.4705
4,5.003568,0,148.943714,1,8,5,24805311,5.008170,115.4929


In [107]:
df_Figure5.to_csv('figure5_pred.csv')